In [ ]:
#default_exp utils.tmy

In [ ]:
#export
from capetools.imports import *
import dateutil
import pandas as pd
import pytz

# TMY: Typical Meteorological Year
> Reading different types of tmy files

In [ ]:
#export
def read_tmy(filename: File, columns:Optional[list]=None, coerce_year:Optional[int]=None):
    "Reads TMY files with dates on 5 first cols."
    if columns is None:
        columns = ['ghi', 'dni', 'dhi','temp_air', 'temp_dew', 
                   'humidity', 'air_pressure','wind_speed', 'wind_dir', 'snow']
    with open(filename, 'r') as csvdata:
        # read in file metadata, advance buffer to second line
        firstline = csvdata.readline().rstrip('\n').split(",")
        secondline = csvdata.readline().rstrip('\n').split(",")
        meta = dict(zip(firstline,secondline))
        meta['Latitude'] = float(meta['Latitude'])
        meta['Longitude'] = float(meta['Longitude'])
        meta['Elevation'] = float(meta['Elevation'])
        meta['Time Zone'] = int(meta['Time Zone'])
        fixed_tz = pytz.FixedOffset(float(meta['Time Zone']) * 60)
        def _parsedate(ymdh, year=None):
            "parser to get a coerced year from a tmy file"
            ymdh = pd.datetime.strptime(ymdh, '%Y %m %d %H %M')
            if year is not None:
                ymdh = ymdh.replace(year=year)
            return ymdh  
        data = pd.read_csv(csvdata, header=0, 
                           index_col='datetime', 
                           parse_dates={'datetime': [0,1,2,3,4]}, 
                           date_parser=lambda x: _parsedate(x, year=coerce_year)
                          )
        data.columns = columns
        return meta, data.tz_localize(fixed_tz)

In [ ]:
#export
def read_pvgis(filename, coerce_year=None):
    "Utility function to read PVGIS tmy files"
    with open(filename, 'r') as csvdata:
        # read in file metadata, advance buffer to second line
        gps_data = {}
        for _ in range(3):
            line = csvdata.readline()
            attribute = line.rstrip('\n').split(" ")
            gps_data[attribute[0][0:-1]] = float(attribute[1])

        months_year = pd.read_csv(csvdata, nrows=12, engine='python')
    
        def _parsedate(ymdh, year=None):
            "parser to get a coerced year from a tmy file"

            date = dateutil.parser.parse(ymdh, dayfirst=True)
            if year is not None:
                date = date.replace(year=year)
            return date

        date_col = 'datetime'
        col_names = ['datetime','temp_air','humidity','ghi', 'dni', 'dhi',
                    'infrared_downwars', 'wind_speed', 'wind_dir','air_pressure']
        data = pd.read_csv(csvdata, 
                           header=0,
                           names=col_names,
                           parse_dates=True,
                           date_parser=lambda x: _parsedate(x, year=coerce_year),
                           index_col=date_col)
        return gps_data, months_year, data


# Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.core.ipynb.
Converted 01_utils.pandas_hacks.ipynb.
Converted 03_utils.matlab_io.ipynb.
Converted 03_utils.tmy.ipynb.
Converted 04_utils.missing.ipynb.
Converted 05_utils.plot.ipynb.
Converted 06_modelling.mybifacialvf.ipynb.
Converted 07_modelling.mypvfactors.ipynb.
Converted index.ipynb.
